# Introduction to LLM-Compressor 🗜️

[llm-compressor](https://github.com/vllm-project/llm-compressor) is the production quantization toolkit from the vLLM project. It provides a simple `oneshot` API for applying post-training quantization.

In this notebook, you'll:
1. Set up llm-compressor
2. Quantize a small model using the `oneshot` API
3. Compare the file size before and after quantization

The example included below is designed to run on CPU rather than GPU.  It's a simple demonstration only of the process involved with compression.  Real world compression for enterprise use cases will involve much larger models and require multiple GPUs.

## 0. Setup

Install dependencies. This notebook runs on CPU.

In [ ]:
!pip install -U "llmcompressor" "transformers>=4.46.0" "datasets" accelerate torch --quiet
print("Setup complete.")

In [ ]:
import pathlib
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_ID = "Qwen/Qwen2-0.5B-Instruct"

---
## 1. The Oneshot API

The `oneshot` function is llm-compressor's main interface for quantization:

```python
oneshot(
    model="model-name",           # HuggingFace model ID
    dataset="dataset-name",       # Calibration dataset
    recipe=recipe,                # Quantization configuration
    output_dir="./output",        # Where to save
    num_calibration_samples=512,  # Samples for calibration
    max_seq_length=4096,          # Sequence length
)
```

---
## 2. Quantize the Model

We'll quantize `Qwen/Qwen2-0.5B-Instruct` using a `GPTQModifier` recipe with W8A8 scheme.

In [ ]:
from llmcompressor import oneshot
from llmcompressor.modifiers.quantization import GPTQModifier

OUTPUT_DIR = "qwen2-0.5b-W8A8"

# Define the quantization recipe
recipe = [
    GPTQModifier(
        scheme="W8A8",        # 8-bit weights and activations
        targets="Linear",     # Quantize Linear layers
        ignore=["lm_head"],   # Keep output layer in full precision
    ),
]

# Run quantization
oneshot(
    model=MODEL_ID,
    dataset="wikitext",
    dataset_config_name="wikitext-2-raw-v1",
    recipe=recipe,
    output_dir=OUTPUT_DIR,
    max_seq_length=64,
    num_calibration_samples=8,
)

print(f"\nQuantized model saved to: {OUTPUT_DIR}")

### GPTQModifier Parameters

| Parameter | Purpose | Common Values |
|-----------|---------|---------------|
| `scheme` | Precision format | `W4A16`, `W8A16`, `W8A8` |
| `targets` | Layer types to quantize | `"Linear"` |
| `ignore` | Layers to keep in FP16 | `["lm_head"]` |
| `group_size` | Weights per scale factor | `128` (default), `64` |

---
## 3. Compare File Sizes

Let's compare the size of the quantized model to the original.

In [ ]:
def folder_size(path: str) -> int:
    """Get total size of files in a directory"""
    p = pathlib.Path(path)
    if not p.exists():
        return 0
    return sum(f.stat().st_size for f in p.rglob("*") if f.is_file())

def format_size(bytes: int) -> str:
    """Format bytes as human-readable string"""
    if bytes < 1024:
        return f"{bytes} B"
    elif bytes < 1024**2:
        return f"{bytes/1024:.1f} KB"
    elif bytes < 1024**3:
        return f"{bytes/1024**2:.1f} MB"
    return f"{bytes/1024**3:.2f} GB"

In [ ]:
# Get quantized model size
size_q = folder_size(OUTPUT_DIR)

print("Model Size Comparison")
print("=" * 40)
print(f"Original (FP16):    ~1.0 GB")
print(f"Quantized (W8A8):   {format_size(size_q)}")

In [ ]:
# List the quantized model files
!ls -lh {OUTPUT_DIR}/

---
## Summary

You've learned:

✅ The `oneshot` API is llm-compressor's main interface for quantization

✅ `GPTQModifier` configures the quantization algorithm and scheme

✅ Key parameters: `scheme`, `targets`, `ignore`, `group_size`

✅ Quantization significantly reduces model size

**Next:** Continue to [Advanced Quantization](./2-advanced-quantization.ipynb) to explore different schemes and compare trade-offs.